## Get Data

To download all match data from static dataset. Run `python lol/download_static_data.py` this will store all static data into `lol/static_matches.data`.

#### Load saved data

In [3]:
import utils

In [4]:
static_matches = utils.load_data('lol/static_matches.data')

## Build a network

In [5]:
import networkx as nx
import time
from collections import deque

In [6]:
import algos
reload(algos)

<module 'algos' from 'algos.pyc'>

In [7]:
from lol import riot_games_api
reload(riot_games_api)

<module 'lol.riot_games_api' from 'lol/riot_games_api.pyc'>

In [8]:
rg = riot_games_api.RiotGames('lol/riot_games_api.key')

#### Find all team mates for one summoner

In [9]:
# Seed node
summonerId = static_matches[0]['participantIdentities'][0]['player']['summonerId']

In [10]:
del static_matches

In [22]:
# Initialize -- Starting from seed
matches = {}  # Store data for all matches
discovered_summonerIds = {summonerId: True}  # Track discovered nodes
g = nx.Graph()

max_hop = 2  # Used as a stopping condition for Breadth First Search

bfs_queue = deque([{'summonerId': summonerId, 'hop': 0}])  # Queue for breadth first search
hop = 0

In [1]:
def load_state(num):
    matches = utils.load_data('matches_' + str(num))
    discovered_summonerIds = utils.load_data('discovered_summonerIds_' + str(num))
    g = utils.load_data('g_' + str(num))
    max_hop = utils.load_data('max_hop_' + str(num))
    bfs_queue = utils.load_data('bfs_queue_' + str(num))
    hop = utils.load_data('hop_' + str(num))
    return matches, discovered_summonerIds, g, max_hop, bfs_queue, hop

In [ ]:
# Optional: Initialize from saved state
matches, discovered_summonerIds, g, max_hop, bfs_queue, hop = load_state(0)

In [2]:
def save_state(num, matches, discovered_summonerIds, g, max_hop, bfs_queue, hop):
    # Save state
    print "Saving state"
    utils.save_data(matches, 'matches_' + str(num))
    utils.save_data(discovered_summonerIds, 'discovered_summonerIds_' + str(num))
    utils.save_data(g, 'g_' + str(num))
    utils.save_data(max_hop, 'max_hop_' + str(num))
    utils.save_data(bfs_queue, 'bfs_queue_' + str(num))
    utils.save_data(hop, 'hop_' + str(num))

In [21]:
checkpoint_num = 0
loop_count = 0

In [ ]:
while (len(bfs_queue)>0) and (hop <= max_hop):
    
    loop_count += 1
    
    # Dequeue next summonerId
    next_item_in_queue = bfs_queue.popleft()
    summonerId = next_item_in_queue['summonerId']
    hop = next_item_in_queue['hop']
    
    # Get match ids for current node
    matchIds = rg.get_matchIds_by_summoner(summonerId)
    num_matches = len(matchIds)
    
    print ""
    print "SummonerId %d at hop %d has %d matches." % (summonerId, hop, num_matches)
    
    # Loop through all matches and add all members to the network
    for i, matchId in enumerate(matchIds):
        print "\rMatch %d out of %d [%0.1f%%]" % (i+1, num_matches, (i+1)/float(num_matches)*100),
        
        if matchId in matches:  # skip loop if match info already retreived
            continue

        # Get full match data and extract team members
        try:
            match = rg.get_match(matchId)
        except:
            print "Error, skipping matchId: %d" % matchId
            continue
            
        team_memberIds = rg.get_summonerIds_by_match(match=match, team_with=summonerId)

        # Add team members to graph as a clique, with matchId info on edges
        g = algos.add_clique_with_weights(g, team_memberIds, edge_attr={'matchId': matchId}) 

        # Add new memberIds to queue
        for i in team_memberIds:
            if i not in discovered_summonerIds:
                bfs_queue.append({'summonerId': i, 'hop': hop+1})
                discovered_summonerIds[i] = True

        # Add match data to matches dict
        match = utils.list_of_dict_to_dict([match], 'matchId')
        matches.update(match)
        
        # Sleep to stay under the API data rate limit
        time.sleep(1.2)        
    
    if loop_count % 100 == 0:
        # Save data every 1000 summonerIds
        save_state(checkpoint_num)
        checkpoint_num += 1



SummonerId 51666047 at hop 0 has 478 matches.
Match 170 out of 478 [35.6%] Status code: 500, 	 Retry count: 1
Retrying in 4.00 seconds
Match 237 out of 478 [49.6%] Status code: 500, 	 Retry count: 1
Retrying in 4.00 seconds
Match 478 out of 478 [100.0%] 
SummonerId 71251277 at hop 1 has 167 matches.
Match 167 out of 167 [100.0%] 
SummonerId 31963680 at hop 1 has 1811 matches.
Match 398 out of 1811 [22.0%] Status code: 500, 	 Retry count: 1
Retrying in 4.00 seconds
Match 1129 out of 1811 [62.3%] Status code: 500, 	 Retry count: 1
Retrying in 4.00 seconds
Match 1718 out of 1811 [94.9%] Status code: 404, 	 Retry count: 1
Retrying in 4.00 seconds
Status code: 404, 	 Retry count: 2
Retrying in 4.00 seconds
Status code: 404, 	 Retry count: 3
Retrying in 4.00 seconds
Status code: 404, 	 Retry count: 4
Retrying in 4.00 seconds
Status code: 404, 	 Retry count: 5
Retrying in 4.00 seconds
Error, skipping matchId: 1467681944
Match 1811 out of 1811 [100.0%] 
SummonerId 58202324 at hop 1 has 807 ma

In [ ]:
len(matches)

In [ ]:
len(bfs_queue)

In [ ]:
print "Network as %d nodes and %d edges" % (len(g.nodes()), len(g.edges()))